In [ ]:
import os
base_folder = "/Users/aishwaryamalgonde/Aishwarya/nanonets/data"
os.chdir(base_folder)
os.getcwd()

In [ ]:
import xml.etree.ElementTree as ET
import glob
import pandas as pd
import statistics
import math
import cv2
import matplotlib.pylab as plt
import shutil

In [ ]:
def check_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
#         print('Creating directory -', directory)

# XML to PD dataframe

In [ ]:
xml_name = '1.1.xml'
tree = ET.parse(xml_name)
root = tree.getroot()
counter = 0
for member in root.findall('object'):
    counter += 1
#     if int(member[3].text) == 1:
#         name = member[0].text+':rotated'
#     else:
#         name = member[0].text
        
    pos_n = int((member[0].text).split('_')[0])
    typec = (member[0].text).split('_')[1]
    value = (xml_name,
             int(member[4][0].text),
             int(member[4][1].text),
             int(member[4][2].text),
             int(member[4][3].text),
             pos_n,
             typec,
             int(member[3].text))
    print(value)
print(counter)

In [ ]:
def xml_to_csv(path):
    xml_list = []
#     xml_list_2 = []
    counter = 0
    for xml_file in glob.glob(path + '/*.xml'):
        xml_file_name = xml_file.split("/")[-1].split('.xml')[0]+'.jpg'
        tree = ET.parse(xml_file)
        root = tree.getroot()
        counter += 1
        print(xml_file)
        for member in root.findall('object'):
            if int(member[3].text) == 1:
                name = member[0].text+':rotated'
            else:
                name = member[0].text
            pos_n = int((member[0].text).split('_')[0])
            typec = (member[0].text).split('_')[1]
            value = (xml_file_name,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text),
                     pos_n,
                     typec,
                     int(member[3].text))
            xml_list.append(value)
#             value_2 = (xml_file_name,
#                  int(member[4][0].text),
#                  int(member[4][1].text),
#                  int(member[4][2].text),
#                  int(member[4][3].text),
#                  name)
#             xml_list_2.append(value_2)
    # column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    column_name = ['filename', 'xmin', 'ymin', 'xmax', 'ymax', 'pos_n', 'typec', 'rotated']
#     column_name_2 = ['filename', 'xmin', 'ymin', 'xmax', 'ymax','class_2']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
#     xml_df_2 = pd.DataFrame(xml_list_2, columns=column_name_2)
    print("Scanned Files :", counter)
    return xml_df#, xml_df_2

In [ ]:
for directory in ['train']:
    working_directory = os.path.join(base_folder, directory)
    xml_df = xml_to_csv(base_folder)
#     xml_df.to_csv(working_directory + '_labels.csv', index=None)
#     xml_df_2.to_csv(working_directory + '_rotated_labels.csv', index=None)

#     data = csv_to_txt(xml_df)
#     data.to_csv(working_directory + '_annotate.txt', header=None, index=None, sep=' ')
    
#     data = csv_to_txt(xml_df_2)
#     data.to_csv(working_directory + '_rotated_annotate.txt', header=None, index=None, sep=' ')

    # xml_df.to_csv('{}_labels.csv'.format(directory), index=None)
    print('Successfully converted xml to csv to txt - {}'.format(directory))

In [ ]:
xml_df.head()

In [ ]:
cols = ['pos_n', 'typec']
xml_df['component'] = xml_df[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

In [ ]:
all_components = (list(set(list(xml_df['component']))))
all_image_name = list(set(list(xml_df['filename'])))
len(all_image_name), len(all_components)

# Averaging position

In [ ]:
avg_position = []
counter = 0
for c in all_components:
#     print(c)
    groupby_component = xml_df.loc[xml_df['component'] == c]
    
    ## Finding average postion
    xmin_avg = int(groupby_component['xmin'].mean())
    ymin_avg = int(groupby_component['ymin'].mean())
    xmax_avg = int(groupby_component['xmax'].mean())
    ymax_avg = int(groupby_component['ymax'].mean())
    
    value = (c, xmin_avg, ymin_avg, xmax_avg, ymax_avg)
    avg_position.append(value)
    counter += 1
#     print(groupby_component)
    
#     if counter>0:
#         break
column_name = ['component', 'xmin_avg', 'ymin_avg', 'xmax_avg', 'ymax_avg']
avg_position_df = pd.DataFrame(avg_position, columns=column_name)
print(counter)
avg_position_df.to_csv('avg_positions_of_components.csv', index=None)
# avg_position

In [ ]:
len(set(list(xml_df['combined'])))

In [ ]:
len(((xml_df['filename']))),len(set(list(xml_df['filename']))),len(set(list(xml_df['pos_n']))),len(set(list(xml_df['typec'])))

# Missing Image - Similarity calculation

In [ ]:
from skimage import measure
import numpy as np

#s = ssim(imageA, imageB)
# s = measure.compare_ssim(imageA, imageB)

In [ ]:
def mse(imageA, imageB):
    # the 'Mean Squared Error' between the two images is the
    # sum of the squared difference between the two images;
    # NOTE: the two images must have the same dimension
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])

    # return the MSE, the lower the error, the more "similar"
    # the two images are
    return err

In [ ]:
type_c = 'h'
pos = 24

In [ ]:
_24_h_df = xml_df.loc[xml_df['pos_n'] == pos]
len(set(_24_h_df['filename']))

In [ ]:
## Finding average postion
xmin_avg = int(_24_h_df['xmin'].mean())
ymin_avg = int(_24_h_df['ymin'].mean())
xmax_avg = int(_24_h_df['xmax'].mean())
ymax_avg = int(_24_h_df['ymax'].mean())

In [ ]:
for i in all_image_name:
#     print(i)
    if i not in list(set(list(_24_h_df['filename']))):
        print(i)

# Thresholding for missing components

In [ ]:
# missing_components_list = ['29_i', '30_i']
missing_components_list = ['3_b', '4_b', '33_e', '20_f', '25_f', '24_h', '28_i', '29_i', '30_i', 
                           '45_i', '47_n', '48_n', '11_q', '13_s', '42_z',
                           '53_ac', '55_l', '56_ae', '57_af']
missing_threshold = {}

In [ ]:
from PIL import Image
import imagehash

In [ ]:
for missing_components in missing_components_list:
    component_df = xml_df.loc[xml_df['component'] == missing_components]

    ref_img = cv2.imread('10.jpg')

    position = avg_position_df.loc[avg_position_df['component'] == missing_components]
    xmin_avg = int(position['xmin_avg'].mean())
    ymin_avg = int(position['ymin_avg'].mean())
    xmax_avg = int(position['xmax_avg'].mean())
    ymax_avg = int(position['ymax_avg'].mean())

    ref_crop = ref_img[ymin_avg:ymax_avg, xmin_avg:xmax_avg, :]
    ref_crop_1 = Image.fromarray(ref_crop)
    ref_crop_1_hash = imagehash.average_hash(ref_crop_1)
    
    present = []
    absent = []

    print(missing_components)
    for image_name in all_image_name:
        
        test_img = cv2.imread(image_name)
        test_crop = test_img[ymin_avg:ymax_avg, xmin_avg:xmax_avg, :]
        
        ref_mean = np.mean(ref_crop, axis = 2)
        test_mean = np.mean(test_crop, axis = 2)
        output = abs(np.mean(ref_mean-test_mean))
        
#         output = mse(ref_crop, test_crop)
        
        if image_name not in list(set(list(component_df['filename']))):
            print('Missing_images', image_name) 
            absent.append(output)
        else:
            present.append(output)
    
    threshold = (statistics.mean(absent)+statistics.mean(present))/2
    missing_threshold[missing_components] = threshold

In [ ]:
missing_threshold_copy = missing_threshold
missing_threshold_copy

In [ ]:
import pickle

with open('missing_threshold_mean.txt', 'wb') as handle:
  pickle.dump(missing_threshold_copy, handle)

with open('missing_threshold_mean.txt', 'rb') as handle:
  missing_threshold_read = pickle.loads(handle.read())

print (missing_threshold_copy == missing_threshold_read) # True

In [ ]:
missing_threshold_read['3_b']

In [ ]:
mse(crop_ref_2, crop_ref)

In [ ]:
measure.compare_ssim(crop_ref_2, crop_ref, multichannel=True)

In [ ]:
## Crop regions from above images
image = cv2.imread('2.jpg')
crop_ref_2 = image[ymin_avg:ymax_avg, xmin_avg:xmax_avg, :]
plt.imshow(crop_ref_2)

In [ ]:
## Crop regions from above images
image = cv2.imread('1.jpg')
crop_ref = image[ymin_avg:ymax_avg, xmin_avg:xmax_avg, :]
plt.imshow(crop_ref)

In [ ]:
## Crop regions from above images
image = cv2.imread('15.15.jpg')
crop = image[ymin_avg:ymax_avg, xmin_avg:xmax_avg, :]
plt.imshow(crop)

## Creating component crops

In [ ]:
def rotation_flag(rotated):
    if rotated==0:
        return 'correct'
    else:
        return 'rotated'

In [ ]:
rotated_components = ['6_6', '20_f', '25_f', '28_i', '30_i', '36_i', '45_i', '44_m', 
                      '46_n', '51_n', '47_n', '48_n', '60_o', '13_s', '27_x', '42_z']

In [ ]:
for c in rotated_components:
    component = c
    print(c)
    i_df = xml_df.loc[xml_df['component'] == component]
    print(len(i_df['component']))
    counter = 0
    for index, row in i_df.iterrows():
#         print(row['filename'])
        image = cv2.imread(row['filename'])
        crop = image[row['ymin']:row['ymax'], row['xmin']:row['xmax'], :]


        path = os.path.join(os.getcwd(), 'split_typec', row['component'], rotation_flag(row['rotated']))
        check_dir(path)

        name = row['filename']+'_'+row['component']+'_'+str(row['rotated'])+'.jpg'
        image_path = os.path.join(path, name)

        counter += 1
        cv2.imwrite(image_path, crop)
    print(counter)

## Preparing test vs train folder for each rotated component

In [ ]:
def check_and_create_dir(directory):
    if not os.path.exists(directory):
        print('DIRECTORY DOESNT EXISTS -', directory)
        os.mkdir(directory)
        print()
        print('DIRECTORY CREATED -', directory)
    else:
        shutil.rmtree(directory)
        os.mkdir(directory)
        print('DIRECTORY EXISTS. Deleted. Created New -', directory)
    print()

In [ ]:
base_folder = '/Users/aishwaryamalgonde/Aishwarya/nanonets/data/split_typec'
os.chdir(base_folder)
os.getcwd()

from glob import glob
from random import shuffle

In [ ]:
rotated_components = ['6_6', '20_f', '25_f', '28_i', '30_i', '36_i', '45_i', '44_m', 
                      '46_n', '51_n', '47_n', '48_n', '60_o', '13_s', '27_x', '42_z']

In [ ]:
component = '44_m'

In [ ]:
train_path = os.path.join(component, 'train')
test_path = os.path.join(component, 'test')

check_and_create_dir(train_path)
check_and_create_dir(test_path)

train_path_rotated = os.path.join(component, 'train', 'rotated')
test_path_rotated = os.path.join(component, 'test', 'rotated')

check_and_create_dir(train_path_rotated)
check_and_create_dir(test_path_rotated)

train_path_correct = os.path.join(component, 'train', 'correct')
test_path_correct = os.path.join(component, 'test', 'correct')

check_and_create_dir(train_path_correct)
check_and_create_dir(test_path_correct)

In [ ]:
# rotated
rotated_path = os.path.join(component, 'rotated')
all_images = glob(os.path.join(rotated_path,'*.jpg'))
all_images
for image_path in all_images:
    image = cv2.imread(image_path)
    name = image_path.split('/')[-1]

    path = os.path.join(train_path_rotated, name)
    cv2.imwrite(path, image)
    
    path = os.path.join(test_path_rotated, name)
    cv2.imwrite(path, image)

In [ ]:
# correct
train_n = 18
test_n = 4

correct_path = os.path.join(component, 'correct')
all_images = glob(os.path.join(correct_path,'*.jpg'))
shuffle(all_images)
print(len(all_images))

for image_path in all_images[:train_n]:
    image = cv2.imread(image_path)
    name = image_path.split('/')[-1]

    path = os.path.join(train_path_correct, name)
    cv2.imwrite(path, image)
    
for image_path in all_images[train_n:train_n+test_n]:
    image = cv2.imread(image_path)
    name = image_path.split('/')[-1]
    
    path = os.path.join(test_path_correct, name)
    cv2.imwrite(path, image)

In [ ]:
from collections import Counter
Counter(all_image_name)